### Ноутбук с рецептами работы в Спарк

In [1]:
import os
import sys

In [2]:
import pandas as pd
from scipy import stats

from importlib import reload
import time
from tqdm import tqdm

In [3]:
print(sys.version)

3.11.4 | packaged by conda-forge | (main, Jun 10 2023, 18:08:17) [GCC 12.2.0]


### Загрузка пакетов и данных

In [4]:
import enviserv.dictan
from enviserv.dictan import DictAnalyzer
import pandserv as pds # сервисные функции для пандас и не только

Создание сессии Спарк с подключением к кластеру упаковано в класс SparkApp.  
Обращение с полями датафрейма Спарк- создание псевдонимов - в классе Cols

In [5]:
from sparkserv import SparkApp, Cols

In [6]:
import pyspark
import pyspark.sql.functions as f

In [7]:
print(pyspark.__version__)

3.4.1


In [8]:
spark_app = SparkApp(my_logger_create_level = 'INFO')

INFO:spark_app:spark_master_ip: 172.18.0.2
INFO:spark_app:pyspark version: 3.4.1
INFO:spark_app:starting building spark app object: pyspark-taxi-forecasting
INFO:spark_app:Spark app object built as: <pyspark.sql.session.SparkSession object at 0x7fdca185af10>
INFO:spark_app:==================================================================
INFO:spark_app:Spark object can be accessed as the SparkApp_object.spark property
INFO:spark_app:==================================================================


In [9]:
spark_master_ip = spark_app.get_spark_master_ip()
# print(spark_master_ip)

INFO:spark_app:spark_master_ip: 172.18.0.2


In [10]:
# spark = spark_app.build_spark_app(spark_master_ip=spark_master_ip)
spark = spark_app.spark

In [11]:
%%time
taxi = spark.read.load('/work/data/Taxi_Trips_-_2022.csv', 
                       format='csv', header='true', inferSchema='true')

CPU times: user 15.5 ms, sys: 971 µs, total: 16.5 ms
Wall time: 39.8 s


In [12]:
taxi.take(2)

[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095', Taxi ID='368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='01/01/2022 12:00:00 AM', Trip Seconds=152, Trip Miles=0.1, Pickup Census Tract=None, Dropoff Census Tract=None, Pickup Community Area=None, Dropoff Community Area=None, Fare=3.75, Tips=0.0, Tolls=0.0, Extras=0.0, Trip Total=3.75, Payment Type='Cash', Company='Medallion Leasin', Pickup Centroid Latitude=None, Pickup Centroid Longitude=None, Pickup Centroid Location=None, Dropoff Centroid Latitude=None, Dropoff Centroid Longitude=None, Dropoff Centroid  Location=None),
 Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12', Taxi ID='449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='

Интерфейс ДатаФрейма работает

In [13]:
taxi_rdd = spark.sparkContext.textFile('/work/data/Taxi_Trips_-_2022.csv')

In [14]:
taxi_rdd.take(5)

['Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,Tolls,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid  Location',
 'bcfa19f2539021c054809d4c3993d226996ae095,368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334,01/01/2022 12:00:00 AM,01/01/2022 12:00:00 AM,152,0.1,,,,,3.75,0.00,0.00,0.00,3.75,Cash,Medallion Leasin,,,,,,',
 '2aba69ff015f9ea8e7bff43cab7eddb228f34a12,449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317,01/01/2022 12:00:00 AM,01/01/2022 12:30:00 AM,2360,17.44,,,,8,47.75,0.00,0.00,5.00,52.75,Cash,Flash Cab,,,,41.899602111,-87.633308037,POINT (-87.6333080367 41.899

Интерфейс РДД также работает

In [15]:
%%time
# Вывести схему данных
taxi.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

### Cols- псевдонимы имен полей ДФ

In [16]:
ct = Cols(taxi)

ct.get_aliases()

{'trip_i': 'Trip ID',
 'taxi_i': 'Taxi ID',
 'trip_st': 'Trip Start Timestamp',
 'trip_et': 'Trip End Timestamp',
 'trip_s': 'Trip Seconds',
 'trip_m': 'Trip Miles',
 'pickup_ct': 'Pickup Census Tract',
 'dropoff_ct': 'Dropoff Census Tract',
 'pickup_ca': 'Pickup Community Area',
 'dropoff_ca': 'Dropoff Community Area',
 'fare': 'Fare',
 'tips': 'Tips',
 'tolls': 'Tolls',
 'extras': 'Extras',
 'trip_t': 'Trip Total',
 'payment_t': 'Payment Type',
 'company': 'Company',
 'pickup_cl': 'Pickup Centroid Latitude',
 'pickup_clon': 'Pickup Centroid Longitude',
 'pickup_cloc': 'Pickup Centroid Location',
 'dropoff_cl': 'Dropoff Centroid Latitude',
 'dropoff_clon': 'Dropoff Centroid Longitude',
 'dropoff_cloc': 'Dropoff Centroid  Location'}

In [17]:
# Вывод значения - полного имени поля для псевдонима
ct.trip_i

'Trip ID'

### Пандас- показать полное содержимое ячеек столбцов

In [18]:
pd.set_option('display.max_colwidth', None)  # Показать полное содержимое ячеек столбцов

In [19]:
table_descr = pd.read_csv('/work/data/table_descr.csv',sep=';',index_col='Column Name')

table_descr

Description  \
Column Name                                                                                                                                                                                                                 
Trip ID                                                                                                                                                                                 A unique identifier for the trip.   
Taxi ID                                                                                                                                                                                 A unique identifier for the taxi.   
Trip Start Timestamp                                                                                                                                             When the trip started rounded to the nearest 15 minutes.   
Trip End Timestamp                                                                                                                                                 When the trip ended rounded to the nearest 15 minutes.   
Trip Seconds                                                                                                                                                                                 Time of the trip in seconds.   
Trip Miles                                                                                                                                                                                 Distance of the trip in miles.   
Pickup Census Tract                                     The Census Tract where the trip began. For privacy  this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.   
Dropoff Census Tract                                    The Census Tract where the trip ended. For privacy  this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.   
Pickup Community Area                                                                                                   The Community Area where the trip began. This column will be blank for locations outside Chicago.   
Dropoff Community Area                                                                                                  The Community Area where the trip ended. This column will be blank for locations outside Chicago.   
Fare                                                                                                                                                                                               The fare for the trip.   
Tips                                                                                                                                                      The tip for the trip. Cash tips generally will not be recorded.   
Tolls                                                                                                                                                                                             The tolls for the trip.   
Extras                                                                                                                                                                                        Extra charges for the trip.   
Trip Total                                                                                                                                                      Total cost of the trip the total of the previous columns.   
Payment Type                                                                                                                                                                                Type of payment for the trip.   
Company                                                                                                                                                                                                 The taxi company.   
Pickup 

In [20]:
pd.reset_option('display.max_colwidth')  # Сброс максимальной ширины столбцов к значению по умолчанию

In [21]:
# %%time
# describe_result = taxi.describe().toPandas()

# time cost i5 (7g), 4 workers (x) 2gb RAM 
# CPU times: user 352 ms, sys: 32.2 ms, total: 384 ms
# Wall time: 2min 7s

In [22]:
# output_file_path = '/work/data/2022_describe.csv'
# describe_result.to_csv(output_file_path, index=False) #transpose().

In [23]:
dr = pd.read_csv('/work/data/2022_describe.csv',sep=',',index_col='summary')

In [24]:
drf = dr.copy()
pds.get_df_formated(drf, '`',2, 10)

In [25]:
drf.transpose()

summary                         count               mean      stddev  \
Trip ID                     6`382`425                nan         nan   
Taxi ID                     6`382`425                nan         nan   
Trip Start Timestamp        6`382`425                nan         nan   
Trip End Timestamp          6`382`213                nan         nan   
Trip Seconds                6`380`960           1`198.21    1`895.66   
Trip Miles                  6`382`369               6.19        8.00   
Pickup Census Tract         2`623`831  17`031`468`160.38  368`945.90   
Dropoff Census Tract        2`675`331  17`031`411`846.86  345`773.49   
Pickup Community Area       5`868`572              32.35       25.20   
Dropoff Community Area      5`748`741              25.84       20.93   
Fare                        6`378`889              21.73       49.42   
Tips                        6`378`889               2.75        4.08   
Tolls                       6`378`889               0.02        7.66   
Extras                      6`378`889               2.16       21.75   
Trip Total                  6`378`889              26.83       56.96   
Payment Type                6`382`425                nan         nan   
Company                     6`382`425                nan         nan   
Pickup Centroid Latitude    5`870`874              41.90        0.06   
Pickup Centroid Longitude   5`870`874             -87.69        0.10   
Pickup Centroid Location    5`870`874                nan         nan   
Dropoff Centroid Latitude   5`784`494              41.89        0.06   
Dropoff Centroid Longitude  5`784`494             -87.66        0.07   
Dropoff Centroid  Location  5`784`494                nan         nan   

summary                                min             max  
Trip ID                         000000bb18      ffffff1aae  
Taxi ID                         0041f8f0c9      fff84aa08a  
Trip Start Timestamp            01/01/2022      12/31/2022  
Trip End Timestamp              01/01/2022      12/31/2022  
Trip Seconds                             0          86`341  
Trip Miles                               0        2`967.54  
Pickup Census Tract         17`031`010`100  17`031`980`100  
Dropoff Census Tract        17`031`010`100  17`031`980`100  
Pickup Community Area                    1              77  
Dropoff Community Area                   1              77  
Fare                                     0        9`999.75  
Tips                                     0             496  
Tolls                                    0        6`666.66  
Extras                                   0        8`888.88  
Trip Total                               0        9`999.75  
Payment Type                          Cash         Unknown  
Company                         24 Seven T       U Taxicab  
Pickup Centroid Latitude             41.65           42.02  
Pickup Centroid Longitude           -87.91          -87.53  
Pickup Centroid Location        POINT (-87      POINT (-87  
Dropoff Centroid Latitude            41.66           42.02  
Dropoff Centroid Longitude          -87.91          -87.53  
Dropoff Centroid  Location      POINT (-87      POINT (-87

In [26]:
val = int(dr.transpose()['count'].astype(float).max())

In [27]:
print(pds.gvf(val))

6'382'425


In [28]:
%%time
row_count = taxi.count()

# time cost i5 (7g), 4 workers (x) 2gb RAM
# CPU times: user 5.85 ms, sys: 871 µs, total: 6.72 ms
# Wall time: 11.8 s

CPU times: user 0 ns, sys: 8.62 ms, total: 8.62 ms
Wall time: 12.1 s


In [29]:
print(pds.gvf(row_count))

6'382'425


### Пандас - cпособы преобразования object столбца

In [30]:
# способ преобразования object столбца и
# фильтрации по стоблцу с использование to_numeric
drt = dr.transpose()
drtf = drt[pd.to_numeric(drt['count'], errors='coerce') > row_count*0.99][['count']]
# отображение отформатированных данных
pds.get_df_formated(drtf,'`',2, 10)
print('==================',pds.gvf(row_count))
drtf

================== 6'382'425


summary                   count
Trip ID               6`382`425
Taxi ID               6`382`425
Trip Start Timestamp  6`382`425
Trip End Timestamp    6`382`213
Trip Seconds          6`380`960
Trip Miles            6`382`369
Fare                  6`378`889
Tips                  6`378`889
Tolls                 6`378`889
Extras                6`378`889
Trip Total            6`378`889
Payment Type          6`382`425
Company               6`382`425

In [31]:
# способ преобразования object столбца и
# фильтрации по стоблцу с использование astype
# 
# можно сразу применять тип и фильтровать, но значения в поле не поменяют тип
drtf1 = dr.transpose()[dr.transpose()['count'].astype(int) > -1][['count']]
# 
# если нужны вычисления с полем, то надо создать дф
drt1 = dr.transpose()
drt1['count'] = drt1['count'].astype(float)
drt1['not_null']=drt1['count']/row_count
# drt1[['count','not_null']]

In [32]:
drtf1 = drt1[drt1['count'] > -1][['count','not_null']]
# display(drtf1)
# отображение отформатированных данных
pds.get_df_formated(drtf1,'`',2, 10)
display(drtf1)
print('================================')
print('==total rows count==',pds.gvf(row_count))
print('================================')
print('=== columns count======',int(len(drtf1)))
print('================================')

summary                         count not_null
Trip ID                     6`382`425        1
Taxi ID                     6`382`425        1
Trip Start Timestamp        6`382`425        1
Trip End Timestamp          6`382`213     1.00
Trip Seconds                6`380`960     1.00
Trip Miles                  6`382`369     1.00
Pickup Census Tract         2`623`831     0.41
Dropoff Census Tract        2`675`331     0.42
Pickup Community Area       5`868`572     0.92
Dropoff Community Area      5`748`741     0.90
Fare                        6`378`889     1.00
Tips                        6`378`889     1.00
Tolls                       6`378`889     1.00
Extras                      6`378`889     1.00
Trip Total                  6`378`889     1.00
Payment Type                6`382`425        1
Company                     6`382`425        1
Pickup Centroid Latitude    5`870`874     0.92
Pickup Centroid Longitude   5`870`874     0.92
Pickup Centroid Location    5`870`874     0.92
Dropoff Centroid Latitude   5`784`494     0.91
Dropoff Centroid Longitude  5`784`494     0.91
Dropoff Centroid  Location  5`784`494     0.91

==total rows count== 6'382'425
=== columns count====== 23


### take. Отбор только значений элементов строк (из объектов Row)

In [33]:
%%time
taxi.take(1)

CPU times: user 14.4 ms, sys: 453 µs, total: 14.9 ms
Wall time: 321 ms


[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095', Taxi ID='368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='01/01/2022 12:00:00 AM', Trip Seconds=152, Trip Miles=0.1, Pickup Census Tract=None, Dropoff Census Tract=None, Pickup Community Area=None, Dropoff Community Area=None, Fare=3.75, Tips=0.0, Tolls=0.0, Extras=0.0, Trip Total=3.75, Payment Type='Cash', Company='Medallion Leasin', Pickup Centroid Latitude=None, Pickup Centroid Longitude=None, Pickup Centroid Location=None, Dropoff Centroid Latitude=None, Dropoff Centroid Longitude=None, Dropoff Centroid  Location=None)]

In [34]:
taxi.take(3)[:2]

[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095', Taxi ID='368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='01/01/2022 12:00:00 AM', Trip Seconds=152, Trip Miles=0.1, Pickup Census Tract=None, Dropoff Census Tract=None, Pickup Community Area=None, Dropoff Community Area=None, Fare=3.75, Tips=0.0, Tolls=0.0, Extras=0.0, Trip Total=3.75, Payment Type='Cash', Company='Medallion Leasin', Pickup Centroid Latitude=None, Pickup Centroid Longitude=None, Pickup Centroid Location=None, Dropoff Centroid Latitude=None, Dropoff Centroid Longitude=None, Dropoff Centroid  Location=None),
 Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12', Taxi ID='449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='

In [35]:
%%time
# Отбор значений для одного элемента объекта Row (одно поле)
[row[ct.trip_i] for row in taxi.take(3)[:2]]

CPU times: user 5.4 ms, sys: 579 µs, total: 5.98 ms
Wall time: 403 ms


['bcfa19f2539021c054809d4c3993d226996ae095',
 '2aba69ff015f9ea8e7bff43cab7eddb228f34a12']

In [36]:
# список словарей
[{field: row[field] for field in [ct.trip_i, ct.taxi_i]} for row in taxi.take(3)[:2]]

[{'Trip ID': 'bcfa19f2539021c054809d4c3993d226996ae095',
  'Taxi ID': '368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334'},
 {'Trip ID': '2aba69ff015f9ea8e7bff43cab7eddb228f34a12',
  'Taxi ID': '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317'}]

In [37]:
# список списков для нескольких полей, несколько проходов
[[row[ct.trip_i] for row in taxi.take(3)[:2]], [row[ct.taxi_i] for row in taxi.take(3)[:2]]]

[['bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'],
 ['368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317']]

In [38]:
# список кортежей, один проход
list(zip(*[[row[ct.trip_i], row[ct.taxi_i]] for row in taxi.take(3)[:2]]))

[('bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 ('368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317')]

In [39]:
# список списков - один проход
[list(t) for t in zip(*[[row[ct.trip_i], row[ct.taxi_i]] for row in taxi.take(3)[:2]])]

[['bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'],
 ['368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317']]

In [40]:
# список списков - произвольный список полей один проход.
fields = [ct.trip_i, ct.taxi_i]

[list(t) for t in zip(*[[row[field] for field in fields] for row in taxi.take(3)[:2]])]

[['bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'],
 ['368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317']]

### Использование zip(*...) и генератора списка

***Использование `zip(*...)` и генератора списка*** для преобразования кортежей в списки имеет минимальные накладные расходы. Эти операции довольно быстры, так как они выполняются в памяти и не требуют значительных вычислительных ресурсов.

Вот краткий обзор того, как работают эти преобразования:

**zip(*...)**: Это операция, которая транспонирует список списков. Она создает итератор, который группирует элементы из каждого из входных списков. Это делается за один проход по данным, что делает его очень эффективным.
Генератор списка: Преобразование каждого кортежа в список с помощью генератора списка `[list(t) for t in ...]` также выполняется за один проход по данным и выполняет простое преобразование типов. Это эффективно и не требует значительных вычислительных затрат.  
Для небольших наборов данных (несколько десятков или сотен элементов) накладные расходы будут практически незаметными. Для больших наборов данных, таких как миллионы элементов, накладные расходы могут стать более заметными, но по-прежнему будут минимальными по сравнению с другими более затратными операциями (например, ввод-вывод на диск или сложные вычисления).

Чтобы продемонстрировать минимальные накладные расходы, давайте рассмотрим пример с измерением времени:

In [41]:
# Пример данных
rows = [{ct.trip_i: f'trip_id_{i}', ct.taxi_i: f'taxi_id_{i}'} for i in range(1000000)]

# Измеряем время выполнения преобразования
start_time = time.time()
[list(t) for t in zip(*[[row[ct.trip_i], row[ct.taxi_i]] for row in rows])]
end_time = time.time()

print("Время выполнения: ", end_time - start_time)

Время выполнения:  3.3904318809509277


В большинстве случаев такие преобразования в Python очень эффективны, и вам не стоит беспокоиться о них, если нет необходимости оптимизировать производительность до предела.

### select. Отбор значений строк

In [42]:
taxi.select(ct.trip_i).take(4)

[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095'),
 Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef')]

In [43]:
taxi.select(ct.trip_i).take(4)[1:3]

[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace')]

In [44]:
# список списков - произвольный список полей один проход.
fields = [ct.trip_i, ct.taxi_i]

[list(t) for t in zip(*[[row[field] for field in fields] \
                        for row in taxi.select(ct.trip_i,ct.taxi_i).take(4)[1:3]])]

[['2aba69ff015f9ea8e7bff43cab7eddb228f34a12',
  '54d812a0b88f8f9707825261014b3563a0a60ace'],
 ['449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317',
  'f98ae5e71fdda8806710af321dce58002146886c013f411be2cd336e0c224e78ed85626eeee462f0d1f5e6b29a030514b95f7b1b8ca59888a10b52dfe55ddf99']]

### Отбор строк через фильтрацию по номерам

#### monotonically_increasing_id для ДФ

In [45]:
from pyspark.sql.functions import monotonically_increasing_id

In [46]:
%%time
taxi.select(ct.trip_i).filter((monotonically_increasing_id() >= 1) \
                              & (monotonically_increasing_id() <= 4)).collect()

CPU times: user 5.33 ms, sys: 10.7 ms, total: 16 ms
Wall time: 16.1 s


[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef'),
 Row(Trip ID='f1a650ee419b4e52d766432e6f86eca3457bfb37'),
 Row(Trip ID='040caea96573c5743668b138011bc157c7825e86')]

#### zipWithIndex().filter для РДД

In [47]:
%%time
taxi.select(ct.trip_i).rdd.zipWithIndex().filter(lambda x: 1 <= x[1] <= 4).map(lambda x: x[0]).collect()

CPU times: user 54 ms, sys: 10.3 ms, total: 64.3 ms
Wall time: 46.1 s


[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef'),
 Row(Trip ID='f1a650ee419b4e52d766432e6f86eca3457bfb37')]

In [48]:
%%time
taxi.select(ct.trip_i).rdd.take(4)

CPU times: user 19.3 ms, sys: 9.12 ms, total: 28.4 ms
Wall time: 294 ms


[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095'),
 Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef')]

#### Причины неэффективности RDD Approach

**Монолитический подход DataFrame API:**  
`DataFrame API` в Apache Spark значительно оптимизирован за счет планировщика `Catalyst` и исполняющего механизма `Tungsten`. Эти оптимизации отсутствуют в `RDD API`, что делает операции на RDD в целом менее эффективными.  
`DataFrame API` оптимизирован с использованием `Catalyst optimizer`. Он понимает логику операций и может оптимизировать выполнение, используя проекцию, фильтрацию, и другие оптимизации.
В первом подходе, используя `monotonically_increasing_id, Spark` может оптимизировать фильтрацию на основе индексов и выполнять ее более эффективно.  
**Отсутствие оптимизации в RDD API:**  
`RDD API` не имеет уровня оптимизации, как `DataFrame API`. При использовании RDD, вы обязаны выполнять множество шагов вручную, что делает код менее эффективным.  
В данном случае, `zipWithIndex` создает пару `(элемент, индекс)`, которая потом фильтруется и мапится обратно. Это добавляет значительные накладные расходы на распределенные вычисления и коммуникацию между узлами кластера.  
**Сериализация и Десериализация:**  
Операции с RDD включают сериализацию и десериализацию данных между узлами, что может увеличить время выполнения, особенно на больших наборах данных.

**Как улучшить производительность RDD**  
Хотя `DataFrame API` чаще всего предпочтительнее для большинства операций из-за своих оптимизаций, бывают случаи, когда RDD необходимы. Вот несколько советов, чтобы улучшить производительность RDD:  
Используйте `Partitions Wisely`:  
Убедитесь, что количество партиций соответствует размерам данных и ресурсам кластера.  
`Persistence`:  
Используйте методы persist() или cache() для данных, которые будут многократно использоваться.  
`Avoid Shuffling`:  
Минимизируйте количество операций, требующих перемещения данных между узлами (например, groupByKey, reduceByKey).

**Добавление индексов:**  
`zipWithIndex()` добавляет индексы **к каждому элементу** в RDD. Эта операция может потребовать **полной загрузки** данных и выполнения дополнительного шага для распределения индексов.
**Фильтрация:**  
`filter(lambda x: 1 <= x[1] <= 4)` фильтрует элементы на основе их индексов. Эта операция применима **к каждому элементу** после добавления индексов.

Без `zipWithIndex()` не обойтись, если нужно отбирать элементы по их порядковому номеру, так как RDD по своей природе неупорядочен. Однако, если использовать take и затем отбирать нужные элементы, можно обойтись меньшими затратами.

#### rdd.take(4)[1:]

In [49]:
%%time
taxi.select(ct.trip_i).rdd.take(4)[1:]

CPU times: user 7.78 ms, sys: 8.99 ms, total: 16.8 ms
Wall time: 251 ms


[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef')]

#### limit(5).collect()[1:5]

In [50]:
taxi.select(ct.trip_i).limit(5).collect()[1:5]

[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef'),
 Row(Trip ID='f1a650ee419b4e52d766432e6f86eca3457bfb37')]

### DF API и SQL - cравнение скорости выполнения запросов

In [51]:
%%time
pck = taxi.select(ct.pickup_ct).distinct().collect()

CPU times: user 15.8 ms, sys: 1.52 ms, total: 17.3 ms
Wall time: 15.4 s


In [52]:
%%time
taxi.createOrReplaceTempView("taxis")
query = "SELECT DISTINCT `Pickup Census Tract` FROM taxis"
result = spark.sql(query)
pck2 = result.collect()

CPU times: user 17 ms, sys: 0 ns, total: 17 ms
Wall time: 14.2 s


In [53]:
# Количество испытаний
num_trials = 5

# Время выполнения для первого блока кода: сбор уникальных значений с помощью DataFrame API
pck_times = []
for _ in tqdm(range(num_trials)):
    start_time = time.time()
    pck = taxi.select('Pickup Census Tract').distinct().collect()
    end_time = time.time()
    pck_times.append(end_time - start_time)

# Вывод среднего времени выполнения для первого блока кода
print("Среднее время выполнения для первого блока кода: {:.2f} секунд".format(sum(pck_times) / num_trials))

100%|██████████| 5/5 [01:09<00:00, 13.94s/it]

Среднее время выполнения для первого блока кода: 13.93 секунд


In [54]:
# Время выполнения для второго блока кода: сбор уникальных значений с помощью SQL запроса
result_times = []
for _ in tqdm(range(num_trials)):
    start_time = time.time()
    taxi.createOrReplaceTempView("taxis")
    result = spark.sql("SELECT DISTINCT `Pickup Census Tract` FROM taxis")
    pck2 = result.collect()
    end_time = time.time()
    result_times.append(end_time - start_time)

# Вывод среднего времени выполнения для второго блока кода
print("Среднее время выполнения для второго блока кода: {:.2f} секунд".format(sum(result_times) / num_trials))


100%|██████████| 5/5 [01:10<00:00, 14.09s/it]

Среднее время выполнения для второго блока кода: 14.09 секунд


**DF API по скорости сравнимо с SQL**. При этом SQL выполняется немного быстрее.

### filter. Несколько условий.

#### Разная сортировка: orderBy(taxi[f2].asc(), taxi[f3].desc())

In [55]:
%%time
f2 = 'Pickup Centroid Latitude'
f3 = 'Pickup Centroid Longitude'
geos_pck_not_null = taxi.select(f2, f3).filter(taxi[f2].isNotNull() &
                taxi[f3].isNotNull()).distinct().orderBy(taxi[f2].asc(), taxi[f3].desc()).collect()

CPU times: user 33.2 ms, sys: 0 ns, total: 33.2 ms
Wall time: 16.2 s


In [56]:
geos_pck_not_null[:5]

[Row(Pickup Centroid Latitude=41.651921576, Pickup Centroid Longitude=-87.564929171),
 Row(Pickup Centroid Latitude=41.660136051, Pickup Centroid Longitude=-87.60284764),
 Row(Pickup Centroid Latitude=41.663670652, Pickup Centroid Longitude=-87.540935513),
 Row(Pickup Centroid Latitude=41.663759731, Pickup Centroid Longitude=-87.637304085),
 Row(Pickup Centroid Latitude=41.665167734, Pickup Centroid Longitude=-87.535284961)]

#### Несколько условий. isNull(). isNotNull()

In [57]:
%%time
geos_pck_not_null = taxi.select(f2, f3).filter(taxi[f2].isNull() &
                taxi[f3].isNull()).collect()

CPU times: user 3.57 s, sys: 135 ms, total: 3.7 s
Wall time: 18 s


In [58]:
%%time
geos_pck_not_null = taxi.select(f2, f3).filter(taxi[f2].isNull() & taxi[f3].isNull())

CPU times: user 167 ms, sys: 397 µs, total: 167 ms
Wall time: 179 ms


In [59]:
%%time
print(pds.gvf(geos_pck_not_null.count()))

511'551
CPU times: user 8.1 ms, sys: 696 µs, total: 8.8 ms
Wall time: 13.7 s


In [60]:
%%time
taxi.select(f2, f3).filter(taxi[f2].isNull() &
                taxi[f3].isNotNull()).count()

CPU times: user 14.7 ms, sys: 390 µs, total: 15.1 ms
Wall time: 13.5 s


0

In [61]:
%%time
taxi.select(f2, f3).filter(taxi[f2].isNotNull() &
                taxi[f3].isNull()).count()

CPU times: user 8.65 ms, sys: 0 ns, total: 8.65 ms
Wall time: 13.6 s


0

#### taxi[ct.trip_m]>10

In [62]:
taxi.filter(taxi[ct.trip_m]>10).count()

1777033

In [63]:
long = taxi.filter((taxi[ct.trip_m]>9.9) & (taxi[ct.trip_m]<10.1) 
                   & taxi[ct.pickup_cl].isNotNull() & taxi[ct.dropoff_cl].isNotNull() 
                   & taxi[ct.pickup_clon].isNotNull() & taxi[ct.dropoff_clon].isNotNull()
                  )

In [64]:
long.count()

20855

### withColumn(вычисления..)

In [65]:
long = long.withColumn('Distance',(
                                ((taxi[ct.pickup_cl]-taxi[ct.dropoff_cl])**2 +
                                (taxi[ct.pickup_clon]-taxi[ct.dropoff_clon])**2))**0.5)

In [66]:
%%time
long.select(ct.pickup_cl,ct.dropoff_cl,ct.pickup_clon,ct.dropoff_clon,'Distance').take(10)

CPU times: user 6.08 ms, sys: 9.8 ms, total: 15.9 ms
Wall time: 265 ms


[Row(Pickup Centroid Latitude=41.901206994, Dropoff Centroid Latitude=41.901206994, Pickup Centroid Longitude=-87.676355989, Dropoff Centroid Longitude=-87.676355989, Distance=0.0),
 Row(Pickup Centroid Latitude=41.874005383, Dropoff Centroid Latitude=41.9867118, Pickup Centroid Longitude=-87.66351755, Dropoff Centroid Longitude=-87.663416405, Distance=0.11270646238477244),
 Row(Pickup Centroid Latitude=41.899602111, Dropoff Centroid Latitude=41.839086906, Pickup Centroid Longitude=-87.633308037, Dropoff Centroid Longitude=-87.714003807, Distance=0.10086573913913337),
 Row(Pickup Centroid Latitude=41.857183858, Dropoff Centroid Latitude=41.9867118, Pickup Centroid Longitude=-87.620334624, Dropoff Centroid Longitude=-87.663416405, Distance=0.1365046798204703),
 Row(Pickup Centroid Latitude=41.874005383, Dropoff Centroid Latitude=41.857183858, Pickup Centroid Longitude=-87.66351755, Dropoff Centroid Longitude=-87.620334624, Distance=0.046343595040167315),
 Row(Pickup Centroid Latitude=41

## UDF

#### Пример UDF (с устаревшим способом указания типа UDF)
для возврата значения функции норм.распределения вер-ти для 1М случайных значений

In [67]:
%%time

from scipy import stats

big_df = (
    spark
    .range(0, 1_000_000)
    .withColumn('val',f.rand())
)

big_df.cache()
big_df.show(3)

@f.pandas_udf('double', f.PandasUDFType.SCALAR)
def pandas_pdf(v):
    return pd.Series(stats.norm.pdf(v))

(
    big_df
    .withColumn('probability', pandas_pdf(big_df.val))
    .show(5)
)

+---+-------------------+
| id|                val|
+---+-------------------+
|  0| 0.4933668797568008|
|  1|0.39678568800785763|
|  2| 0.5252939042912325|
+---+-------------------+
only showing top 3 rows



/opt/conda/lib/python3.11/site-packages/pyspark/sql/pandas/functions.py:399: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+---+-------------------+-------------------+
| id|                val|        probability|
+---+-------------------+-------------------+
|  0| 0.4933668797568008| 0.3532271402338872|
|  1|0.39678568800785763| 0.3687420339869859|
|  2| 0.5252939042912325|0.34752962140828875|
|  3| 0.4300346751446521|  0.363708177112155|
|  4| 0.9741441861475683|0.24822567148807081|
+---+-------------------+-------------------+
only showing top 5 rows

CPU times: user 181 ms, sys: 22.9 ms, total: 204 ms
Wall time: 3.16 s


Предупреждение  
`/opt/conda/lib/python3.11/site-packages/pyspark/sql/pandas/functions.py:399: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn`  

связано с устареванием способа указания типа UDF (User Defined Function) в PySpark. Сообщение указывает на то, что начиная с Python 3.6 и Spark 3.0, предпочтительно использовать аннотации типов вместо указания типа UDF через параметр f.PandasUDFType.

Конкретно предупреждение означает, что метод указания типа UDF через f.PandasUDFType.SCALAR будет устаревшим в будущих версиях PySpark, и рекомендуется использовать аннотации типов для Pandas UDF. Это сделано для повышения ясности кода и его поддерживаемости

В `@f.pandas_udf('double', f.PandasUDFType.SCALAR)`
- первый параметр `'double'` указывает тип возвращаемого значения ЮДФ,
- второй параметр - это тип функции. Если возвращается один столбец (`.Series(...)`), используется `.PandasUDFType.SCALAR`, если возвращается несколько столбцов (pandas `DataFrame(...)`), `.PandasUDFType.GROUPED_MAP`

### Пример UDF (аргумент - Series) с аннотацией типов

In [68]:
%%time
from pyspark.sql.types import DoubleType

big_df = (
    spark
    .range(0, 1_000_000)
    .withColumn('val', f.rand())
)

big_df.cache()
big_df.show(3)

@f.pandas_udf(DoubleType())
def pandas_pdf(v: pd.Series) -> pd.Series:
    return pd.Series(stats.norm.pdf(v))

result = big_df.withColumn('pdf', pandas_pdf(f.col('val')))
result.show(3)

+---+-------------------+
| id|                val|
+---+-------------------+
|  0|  0.539275029755421|
|  1| 0.5998223231068963|
|  2|0.10141922575152806|
+---+-------------------+
only showing top 3 rows

+---+-------------------+------------------+
| id|                val|               pdf|
+---+-------------------+------------------+
|  0|  0.539275029755421|0.3449529279226693|
|  1| 0.5998223231068963|0.3332601233123248|
|  2|0.10141922575152806| 0.396895815232463|
+---+-------------------+------------------+
only showing top 3 rows

CPU times: user 11.7 ms, sys: 18.7 ms, total: 30.4 ms
Wall time: 2.76 s


Здесь `DoubleType()` используется вместо `f.PandasUDFType.SCALAR`, и типы указаны непосредственно в аннотациях функции `(v: pd.Series) -> pd.Series`. Это улучшает читаемость кода и соответствует новым стандартам PySpark.

В актуальном способе указания типов для PySpark UDF для определения UDF, которая возвращает несколько столбцов (pandas DataFrame), вы можете использовать аннотацию функции и указание типа возвращаемого значения при помощи PySpark StructType. В этом случае вам понадобится указать схему (schema), описывающую структуру возвращаемого DataFrame.

In [69]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StructType, StructField, DoubleType, StringType

# Определяем схему возвращаемого DataFrame
schema = StructType([
    StructField("col1", DoubleType(), True),
    StructField("col2", StringType(), True)
])

# Определяем Pandas UDF
@pandas_udf(schema)
def my_udf(input_col: pd.Series) -> pd.DataFrame:
    # Ваш код для обработки DataFrame
    result = pd.DataFrame({
        "col1": input_col * 2,
        "col2": input_col.astype(str)
    })
    return result

# Создаем DataFrame
df = spark.createDataFrame(
    [(1.0,), (2.0,), (3.0,)],
    ['input_col']
)

# Применяем UDF к столбцу
df.select(my_udf(df['input_col'])).show()

+-----------------+
|my_udf(input_col)|
+-----------------+
|       {2.0, 1.0}|
|       {4.0, 2.0}|
|       {6.0, 3.0}|
+-----------------+



Pandas UDF в PySpark может принимать несколько pandas Series объектов в качестве аргументов. Для этого необходимо использовать декоратор @pandas_udf и определить схему выходного DataFrame.

In [70]:
# Определяем схему выходного DataFrame
output_schema = StructType([
    StructField('output_col1', DoubleType()),
    StructField('output_col2', DoubleType())
])

# Определяем Pandas UDF с использованием декоратора @pandas_udf
@pandas_udf(output_schema)
def my_udf(input_col1: pd.Series, input_col2: pd.Series) -> pd.DataFrame:
    # Создаем DataFrame из двух входных Series
    output_df = pd.DataFrame({
        'output_col1': input_col1 + input_col2,
        'output_col2': input_col1 * input_col2
    })
    return output_df

# Создаем DataFrame для теста
df = spark.createDataFrame(
    [(1.0, 2.0), (3.0, 4.0), (5.0, 6.0)],
    ['input_col1', 'input_col2']
)

# Применяем Pandas UDF к DataFrame
result_df = df.select(my_udf(df['input_col1'], df['input_col2']))
result_df.show()

+------------------------------+
|my_udf(input_col1, input_col2)|
+------------------------------+
|                    {3.0, 2.0}|
|                   {7.0, 12.0}|
|                  {11.0, 30.0}|
+------------------------------+



### Пример UDF (аргумент - DataFrame) с аннотацией типов

Вы можете передать в UDF аргумент в виде DataFrame, но это будет работать только с UDF типа GROUPED_MAP. В этом случае каждый входной DataFrame будет представлять собой группу данных, по которой будет выполняться UDF. Для этого типа UDF вам необходимо определить схему входного DataFrame и схему выходного DataFrame.

In [71]:
# Определяем схему входного DataFrame
input_schema = StructType([
    StructField("col1", DoubleType(), True),
    StructField("col2", DoubleType(), True)
])

# Определяем схему возвращаемого DataFrame
output_schema = StructType([
    StructField("col1", DoubleType(), True),
    StructField("col2", DoubleType(), True),
    StructField("col3", StringType(), True)
])

# Определяем Pandas UDF типа GROUPED_MAP
@pandas_udf(output_schema, f.PandasUDFType.GROUPED_MAP)
def my_udf(pdf: pd.DataFrame) -> pd.DataFrame:
    # Ваш код для обработки DataFrame
    pdf["col3"] = (pdf["col1"] + pdf["col2"]).astype(str)
    return pdf

# Создаем DataFrame
df = spark.createDataFrame(
    [(1.0, 2.0), (3.0, 4.0), (5.0, 6.0)],
    ['col1', 'col2']
)

# Применяем UDF к DataFrame с использованием groupBy и apply
# В данном случае мы используем groupBy().apply() даже без явного группирования
df.groupBy().apply(my_udf).show()

/opt/conda/lib/python3.11/site-packages/pyspark/sql/pandas/group_ops.py:103: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+----+----+----+
|col1|col2|col3|
+----+----+----+
| 5.0| 6.0|11.0|
| 1.0| 2.0| 3.0|
| 3.0| 4.0| 7.0|
+----+----+----+



Возможное предупреждение  
`/opt/conda/lib/python3.11/site-packages/pyspark/sql/pandas/group_ops.py:103: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(`  
говорит о том, что используется устаревший способ `df.groupBy().apply(my_udf).show()`. Pandas UDFs с использованием `applyInPandas` предоставляют более современный и предпочтительный способ обработки pandas DataFrame.

Pandas UDF в режиме GROUPED_MAP не поддерживает передачу нескольких pandas DataFrame в качестве отдельных аргументов. Вместо этого вы должны объединить данные в один DataFrame до передачи в функцию. В режиме SCALAR Pandas UDF может принимать несколько Series в качестве аргументов.

Для использования нескольких pandas DataFrame в Pandas UDF с декоратором @pandas_udf в режиме GROUPED_MAP, вам нужно объединить два DataFrame в один перед тем, как вызывать функцию. PySpark Pandas UDF с типом GROUPED_MAP ожидает один pandas DataFrame в качестве аргумента

In [72]:
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType

# Определяем схему выходного DataFrame
output_schema = StructType([
    StructField('id', IntegerType()),
    StructField('output_col1', DoubleType()),
    StructField('output_col2', DoubleType())
])

# Определяем Pandas UDF с использованием декоратора @pandas_udf
@pandas_udf(output_schema, f.PandasUDFType.GROUPED_MAP)
def my_udf(pdf: pd.DataFrame) -> pd.DataFrame:
    # Создаем выходной DataFrame
    output_df = pd.DataFrame({
        'id': pdf['id'],
        'output_col1': pdf['input_col1'] + pdf['input_col2'],
        'output_col2': pdf['input_col1'] * pdf['input_col2']
    })
    return output_df

# Создаем два DataFrame для теста
df1 = spark.createDataFrame(
    [(1, 1.0), (2, 3.0), (3, 5.0)],
    ['id', 'input_col1']
)

df2 = spark.createDataFrame(
    [(1, 2.0), (2, 4.0), (3, 6.0)],
    ['id', 'input_col2']
)

# Объединяем два DataFrame по столбцу 'id'
df = df1.join(df2, on='id')

# Применяем Pandas UDF к DataFrame с использованием groupBy
result_df = df.groupBy('id').apply(my_udf)
result_df.show()

+---+-----------+-----------+
| id|output_col1|output_col2|
+---+-----------+-----------+
|  1|        3.0|        2.0|
|  2|        7.0|       12.0|
|  3|       11.0|       30.0|
+---+-----------+-----------+



### Сравнение производительности векторизированой и не_ ЮДФ

In [73]:
big_df = (
    spark
    .range(0, 1_000_000)
    .withColumn('val',f.rand())
)

In [74]:
def test_pandas_pdf():
    return (
        big_df
        .withColumn('probability', pandas_pdf(big_df.val))
        .agg(f.count(f.col('probability')))
        .show()
    )

%timeit -n 1 test_pandas_pdf()

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

780 ms ± 464 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [75]:
#row-by-row version with Python-JVM conversion
@f.udf('double')
def pdf(v):
    return float(stats.norm.pdf(v))

def test_pdf():
    return (
        big_df
        .withColumn('probability', pdf(big_df.val))
        .agg(f.count(f.col('probability')))
        .show()
    )

%timeit -n 1 test_pdf()

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|           1000000|
+------------------+

37.2 s ± 4.5 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Результат для векторизированной ЮДФ - 903 ms, против не_ - 34.4 s

Отличие примерно в 40 раз!

### Использование объектов pandas в Pandas UDF

В **Pandas UDF** (User Defined Function) с использование декоратора `@pandas_udf` в Spark DataFrame API, функция pandas_pdf принимает один или несколько pandas Series (DataFrame) объектов и возвращает pandas Series (DataFrame) объект. В этом случае, pandas_pdf принимает Series v (который представляет собой значения из столбца val в DataFrame) и возвращает Series, содержащий значения, вычисленные с использованием функции stats.norm.pdf().

Возвращая `pd.Series(stats.norm.pdf(v))`, мы создаем объект Pandas Series, который содержит значения, вычисленные с помощью функции `stats.norm.pdf()`, примененные к каждому элементу входного Series v. Это необходимо потому, что Spark использует PyArrow для обмена данными между JVM и Python, и PyArrow не может обрабатывать объекты `stats.norm.pdf(v)` напрямую, так как они не являются объектами pandas.

Поэтому, оборачивая результат stats.norm.pdf(v) в pd.Series(), мы создаем объект pandas, который Spark может обработать, и возвращаем его из функции pandas_pdf.

## .cache() .unpersist() - кэш памяти

Когда вы вызываете `big_df.cache()`, вы помещаете DataFrame big_df в **кэш памяти**. Это означает, что DataFrame будет кэширован в памяти и будет доступен для быстрого доступа к данным при последующих операциях.

Однако, если у вас есть ограничения на использование памяти или если кэш становится необходимым для других операций, вы можете явно удалить кэш с помощью метода `unpersist()`.

Это удалит кэш из памяти и освободит ресурсы, занимаемые DataFrame big_df. Если вы не удаляете кэш явно с помощью unpersist(), DataFrame останется в кэше и будет занимать память до тех пор, пока не будет удален Spark контекст или пока явно не будет вызвана операция `unpersist()`

In [76]:
big_df.unpersist()

DataFrame[id: bigint, val: double]

## Apache Arrow Memory & runtime switch

До версии ПиСпарк 2.3 использование ЮДФ в Питоне проигрывало по производительности Скале и Джаве (из-за переключений времени выполнения `runtime` между Питон и JVM).

С версии 2.3 Спарк начал использовать **Apache Arrow Memory** - общее пространство памяти для разных сред окружения, поэтому отпала необходимость и расходны на конвертацию объектов.

Также Arrow хранит в памяти **столбчатые объекты** (columnar objects), что повышает производительность. Используя это, ПиСпарк отрефакторилился, в результате чего мы получили **векторизированные ЮДФ**.

В аспектах программирования и операционных систем (в частности, процессов и потоков), термины "runtime" и "switch runtime" имеют разные значения. Давайте рассмотрим их подробнее:

**Runtime**  
**Программирование:**  
`Runtime` (время выполнения) - это фаза, когда программа исполняется. В это время происходят все динамические операции, такие как выполнение кода, управление памятью, обработка исключений и т.д. Например, в Python есть интерпретатор, который выполняет код во время исполнения программы.
`Runtime Environment` (среда выполнения) - это окружение, в котором исполняется программа. Оно включает в себя библиотеки, интерпретаторы (например, `Python Interpreter`) или виртуальные машины (например, `Java Virtual Machine`).  
**Процессы и потоки ОС:**  
`Runtime` в контексте ОС может также относиться к продолжительности времени, когда процесс или поток активен и выполняется на процессоре. Это время исполнения может включать в себя время, проведенное на процессоре, и время ожидания (например, ожидание ввода-вывода).

**Switch Runtime**  
**Программирование:**  
`Switch Runtime` (переключение времени выполнения) в программировании часто означает смену среды выполнения для кода. Например, если вы хотите исполнять код, написанный на Python, в среде JVM, это может потребовать использование специального инструмента или транслятора, который позволит исполнять Python-код на Java Virtual Machine (например, Jython).  
**Процессы и потоки ОС:**  
`Context Switch` (переключение контекста) - это процесс, при котором операционная система переключается от выполнения одного процесса или потока к другому. Это необходимо для многозадачности, чтобы несколько процессов могли совместно использовать ресурсы процессора.
В данном контексте, "switch runtime" может подразумевать переключение между разными процессами или потоками в ходе их выполнения. То есть операционная система переключает время выполнения между различными задачами для обеспечения параллелизма и эффективного использования процессора.

**Примеры**  
**Программирование:**  
Вы используете Python для разработки, но вам нужно интегрировать код с существующим приложением на Java. Вы используете Jython для "switch runtime" и исполняете Python-код в JVM.
**Процессы и потоки ОС:**  
В многозадачной операционной системе несколько программ работают одновременно. Процессор переключается между задачами (процессами) каждые несколько миллисекунд, обеспечивая иллюзию параллельного выполнения. Это и есть "context switch".

**Резюме**  
`Runtime` (время выполнения): в программировании - фаза исполнения программы; в ОС - время, когда процесс или поток активно выполняется.
`Switch Runtime` (переключение времени выполнения): в программировании - смена среды выполнения для кода; в ОС - переключение контекста между процессами или потоками для многозадачности.

## Объекты Column. Обращение к столбцам через точку

Обращение к полям DataFrame в ПиСпарк через точку, если имена полей содержат несколько слов через пробел, невозможно - необходимо использовать квадратные скобки и строковые индексы.

In [77]:
data = [("John Doe", 30), ("Jane Smith", 25), ("Bob Johnson", 35)]
columns = ["full name", "age"]

# Создание DataFrame
df = spark.createDataFrame(data, schema=columns)

# Обращение к полям через точку
df.show()
df.select("full name").show()
print(df["full name"])
print(df.age)
df.age.show()

+-----------+---+
|  full name|age|
+-----------+---+
|   John Doe| 30|
| Jane Smith| 25|
|Bob Johnson| 35|
+-----------+---+

+-----------+
|  full name|
+-----------+
|   John Doe|
| Jane Smith|
|Bob Johnson|
+-----------+

Column<'full name'>
Column<'age'>


TypeError: 'Column' object is not callable

Для инструкции `df.age.show()` получим ошибку TypeError: 'Column' object is not callable. Обращение к столбцам через точку, как в df.age, может вызвать ошибку TypeError, так как это возвращает объект Column, который не является вызываемым (callable) для метода show().

Объекты `df.age` и `df["full name"]` в PySpark являются объектами типа Column. Column - это абстракция, представляющая столбец данных в DataFrame. Объект Column обеспечивает различные методы для выполнения операций на столбцах данных, таких как вычисления, фильтрация, агрегация и т. д.

Чтобы использовать объект Column, вы можете использовать его методы для создания новых столбцов, применения функций к существующим столбцам и других операций с данными в DataFrame. Например, вы можете использовать объект Column для создания нового столбца, содержащего возраст пассажиров в годах путем деления столбца "age" на 10.

In [ ]:
df = df.withColumn("age_in_decades", df.age/10)
df.show()

Или можете отфильтровать датафрейм по полю

In [ ]:
df.filter(df.age_in_decades > 3).show()